# Exploratory notebook for IMACLIM within MATMat-Trade

## Creating the MatMat-Trade model base on IMACLIM regions and sectors but on EXIOBASE 3 data

In [25]:
# make the necessary imports
import tqdm
import shutil
import warnings
import pandas as pd
import numpy as np
import multiprocessing


from main import Model
from src.utils import footprint_extractor
from src.scenarios import DICT_SCENARIOS
from src.advance import extract_data

In [26]:
# for some reason saved model has issues with the names of regions so we rebuild the model each times


shutil.rmtree('data/models/2015__pxp__IMACLIM_format__ghg')

In [27]:


model = Model(aggregation_name="IMACLIM_format")

#ignore performace warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

Loading data... (may take a few minutes)
Data loaded successfully !


## Creating the scenarios based on IMACLIM scenarios and the projected changes 

In [4]:

scenario_list=["INDC","INDC2020_1000","INDC2030_1000","INDC2020_1600","INDC2020_400","INDC2030_1600","baseline"]

for scenario in tqdm.tqdm(scenario_list[:1]):
    # create 2050 version of each scenarios applying emissivity changes and then technological changes
    model.new_counterfactual("IMACLIM_all_2050_"+scenario, DICT_SCENARIOS["emissivity_IMACLIM"], year=2050,scenario=scenario)
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)
    # create the 2050 version based on production changes observed in IMACLIM (since we work on groos output, we need a reference to apply relative change : x_ref), with name new_name
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["production_change_IMACLIM"], year=2050,scenario="INDC",new_name="IMACLIM_all_prod_change_2050_"+scenario,x_ref=model.iot.x.sort_index())
    
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["consumption_change_imaclim"], year=2050,scenario="INDC",new_name="IMACLIM_all_cons_change_2050_"+scenario)

  0%|          | 0/1 [00:02<?, ?it/s]


KeyboardInterrupt: 

## Getting the emissions per scenario 

### Single process version, might be long to run 

In [ ]:

scenario_list=["INDC","INDC2020_1000","INDC2030_1000","INDC2020_1600","INDC2020_400","INDC2030_1600","baseline"]

start_year=pd.concat([ pd.concat([pd.DataFrame(footprint_extractor(model,region=region),index=[2015]).T  for region in model.regions],
                      keys=model.regions,
                      names=("regions","types")) for scenario in scenario_list],
          keys=scenario_list,
          names=("scenario","regions","types"))

Emissions=pd.concat([start_year for i in range(4)],axis=1,keys=["2015","2050","2050-Prod","2050-Cons"]).sort_index()
Emissions.columns=["2015","2050","2050-Prod","2050-Cons"]

production_based_emissions=pd. concat([ pd.concat([ model.iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"]for i in range(4)],axis=1,keys=["2015","2050","2050-Prod","2050-Cons"]) for scenario in scenario_list],
                                        keys=scenario_list,
                                        names=("scenario","region","sector"),
                                        axis=0).sort_index()


for scenario in tqdm.tqdm(scenario_list):
    # create 2050 version of each scenarios applying emissivity changes and then technological changes
    model.new_counterfactual("IMACLIM_all_2050_"+scenario, DICT_SCENARIOS["emissivity_IMACLIM"], year=2050,scenario=scenario)
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)
    # create the 2050 version based on production changes observed in IMACLIM (since we work on groos output, we need a reference to apply relative change : x_ref), with name new_name
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["production_change_IMACLIM"], year=2050,scenario=scenario,new_name="IMACLIM_all_prod_change_2050_"+scenario,x_ref=model.iot.x.sort_index())
    
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["consumption_change_imaclim"], year=2050,scenario=scenario,new_name="IMACLIM_all_cons_change_2050_"+scenario)
    
    Emissions.loc[(scenario),"2050"]=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    Emissions.loc[(scenario),"2050-Prod"]=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    Emissions.loc[(scenario),"2050-Cons"]=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    production_based_emissions.loc[(scenario),"2050-Cons"]=model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    production_based_emissions.loc[(scenario),"2050-Prod"]=model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    production_based_emissions.loc[(scenario),"2050"]=model.counterfactuals["IMACLIM_all_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    del(model.counterfactuals["IMACLIM_all_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario])
    
    

### Multiple Process versio, should be faster

In [28]:

NBR_PROCESS=4

scenario_list=["INDC","INDC2020_1000","INDC2030_1000","INDC2020_1600","INDC2020_400","INDC2030_1600","baseline"]

start_year=pd.concat([ pd.concat([pd.DataFrame(footprint_extractor(model,region=region),index=[2015]).T  for region in model.regions],
                      keys=model.regions,
                      names=("regions","types")) for scenario in scenario_list],
          keys=scenario_list,
          names=("scenario","regions","types"))

Emissions=pd.concat([start_year for i in range(4)],axis=1,keys=["2015","2050","2050-Prod","2050-Cons"]).sort_index()
Emissions.columns=["2015","2050","2050-Prod","2050-Cons"]

production_based_emissions=pd. concat([ pd.concat([ model.iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"]for i in range(4)],axis=1,keys=["2015","2050","2050-Prod","2050-Cons"]) for scenario in scenario_list],
                                        keys=scenario_list,
                                        names=("scenario","region","sector"),
                                        axis=0).sort_index()


def task(scenario):
    model.new_counterfactual("IMACLIM_all_2050_"+scenario, DICT_SCENARIOS["emissivity_IMACLIM"], year=2050,scenario=scenario)
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)
    # create the 2050 version based on production changes observed in IMACLIM (since we work on groos output, we need a reference to apply relative change : x_ref), with name new_name
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["production_change_IMACLIM"], year=2050,scenario=scenario,new_name="IMACLIM_all_prod_change_2050_"+scenario,x_ref=model.iot.x.sort_index())
    
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["consumption_change_imaclim"], year=2050,scenario=scenario,new_name="IMACLIM_all_cons_change_2050_"+scenario)
    
    ret1=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    ret2=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    ret3=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    ret4=model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    ret5=model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    ret6=model.counterfactuals["IMACLIM_all_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    del(model.counterfactuals["IMACLIM_all_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario])
    
    return ret1,ret2,ret3,ret4,ret5,ret6


pool = multiprocessing.Pool(processes=NBR_PROCESS)
result_list=pool.map(task,scenario_list)

for i in range(len(scenario_list)):
    scenario=scenario_list[i]
    Emissions.loc[(scenario),"2050"],Emissions.loc[(scenario),"2050-Prod"],Emissions.loc[(scenario),"2050-Cons"],production_based_emissions.loc[(scenario),"2050-Cons"],production_based_emissions.loc[(scenario),"2050-Prod"],production_based_emissions.loc[(scenario),"2050"]=result_list[i]
        
    

Problems on sums of columns  region  sector               
AFR     Terrestrial transport     47.043673
MDE     Terrestrial transport    146.430499
RAL     Terrestrial transport     25.875496
dtype: float64
Problems on sums of columns  region  sector               
AFR     Terrestrial transport     333.209566
MDE     Terrestrial transport    4486.348277
RAL     Terrestrial transport     172.151602
dtype: float64
Problems on sums of columns  region  sector               
AFR     Terrestrial transport     237.323465
MDE     Terrestrial transport    3148.991362
RAL     Terrestrial transport     124.123202
dtype: float64
Problems on sums of columns  region  sector               
AFR     Terrestrial transport     282.256477
MDE     Terrestrial transport    6422.740342
RAL     Terrestrial transport     173.155012
dtype: float64
Problems on sums of columns  region  sector               
AFR     Terrestrial transport     169.042045
MDE     Terrestrial transport    2024.383010
RAL     Terrestria

## Exploring the results

In [29]:
#adding IMACLIM direct emissions to the df
IMACLIM_emissions=extract_data(aggregation="IMACLIM_format")[6]

production_based_emissions["2050-IMACLIM"]=IMACLIM_emissions.drop("World",level=1)[2050].rename_axis(index=str.lower)*1e9
production_based_emissions["2015-IMACLIM"]=IMACLIM_emissions.drop("World",level=1)[2015].rename_axis(index=str.lower)*1e9

In [30]:
diff_rapport =((production_based_emissions["2050-IMACLIM"]/production_based_emissions["2015-IMACLIM"])-(production_based_emissions["2050-Prod"]/production_based_emissions["2015"]))

for scenario in scenario_list:
    print(scenario,diff_rapport.loc[scenario].max())

INDC 8.268941087408166e-12
INDC2020_1000 8.035572207631958e-12
INDC2030_1000 7.099210108663101e-12
INDC2020_1600 8.144818153255073e-12
INDC2020_400 6.13709083552294e-12
INDC2030_1600 8.04800670550776e-12
baseline 9.425571434462654e-12


In [31]:
Relative_error_2015=production_based_emissions.groupby(["scenario","region","sector"]).sum().loc["INDC"] 
Relative_error_2015=(Relative_error_2015["2015"]-Relative_error_2015["2015-IMACLIM"])/np.max([Relative_error_2015["2015"],Relative_error_2015["2015-IMACLIM"]],axis=0)
Relative_error_2015=Relative_error_2015*100
pd.concat([Relative_error_2015.loc[region] for region in Relative_error_2015.index.get_level_values("region").unique()],keys=Relative_error_2015.index.get_level_values("region").unique(),axis=1)

region,AFR,BRA,CAN,CHN,CIS,EUR,IND,JAN,MDE,RAL,RAS,USA
sector,,,,,,,,,,,,
Agriculture,-43.520474,36.645700,17.158575,-44.859290,-79.566081,-6.082898,46.911948,-59.574627,-55.962035,-17.970046,29.793671,-0.988413
Air,-45.172227,-23.270153,12.436130,71.661958,-1.527087,21.934999,45.944462,-43.838684,46.889270,-67.679364,39.297675,-5.631817
Coal,44.122147,27.782410,-88.077609,8.506734,-46.838304,-42.924087,96.470371,-22.246558,100.000000,100.000000,99.626550,-51.110549
Construction,84.668945,99.708036,76.663408,96.624980,70.970976,94.460861,95.967109,72.836376,83.099541,98.908128,96.687807,89.505142
Elec,-39.367526,72.760880,-56.968570,38.523421,-96.873208,-44.842949,25.343792,-13.061264,15.577006,-20.202585,7.368399,-19.935929
Gas,49.769182,81.839608,7.038038,97.653085,-7.778965,87.224283,68.865246,100.000000,-30.457425,-2.817599,31.503872,71.159567
Industry,-43.893401,4.640709,-50.285890,22.150305,-73.041125,-28.195911,-13.054238,-59.278069,37.025988,9.404065,12.147867,-52.534332
Liquid fuels,-91.631254,-76.275770,-37.110566,-43.896218,-55.277679,-77.231510,-4.899449,-81.704037,100.000000,100.000000,-60.917096,99.872532
Maritime,-89.689210,-70.072280,46.427379,70.225032,100.000000,41.189947,93.873600,40.498453,-25.642740,-70.193055,-23.795701,-42.158055


In [16]:
Relative_error_2015=production_based_emissions.groupby(["scenario","region","sector"]).sum().loc["INDC"] 
Relative_error_2015=(Relative_error_2015["2050-Prod"]-Relative_error_2015["2050-IMACLIM"])/np.max([Relative_error_2015["2050-Prod"],Relative_error_2015["2050-IMACLIM"]],axis=0)
Relative_error_2015=Relative_error_2015*100
pd.concat([Relative_error_2015.loc[region] for region in Relative_error_2015.index.get_level_values("region").unique()],keys=Relative_error_2015.index.get_level_values("region").unique(),axis=1)

region,AFR,BRA,CAN,CHN,CIS,EUR,IND,JAN,MDE,RAL,RAS,USA
sector,,,,,,,,,,,,
Agriculture,-43.520474,36.645700,17.158575,-44.859290,-79.566081,-6.082898,46.911948,-59.574627,-55.962035,-17.970046,29.793671,-0.988413
Air,-45.172227,-23.270153,12.436130,71.661958,-1.527087,21.934999,45.944462,-43.838684,46.889270,-67.679364,39.297675,-5.631817
Coal,44.122147,27.782410,-88.077609,8.506734,-46.838304,-42.924087,96.470371,-22.246558,100.000000,100.000000,99.626550,-51.110549
Construction,84.668945,99.708036,76.663408,96.624980,70.970976,94.460861,95.967109,72.836376,83.099541,98.908128,96.687807,89.505142
Elec,-39.367526,72.760880,-56.968570,38.523421,-96.873208,-44.842949,25.343792,-13.061264,15.577006,-20.202585,7.368399,-19.935929
Gas,49.769182,81.839608,7.038038,97.653085,-7.778965,87.224283,68.865246,100.000000,-30.457425,-2.817599,31.503872,71.159567
Industry,-43.893401,4.640709,-50.285890,22.150305,-73.041125,-28.195911,-13.054238,-59.278069,37.025988,9.404065,12.147867,-52.534332
Liquid fuels,-91.631254,-76.275770,-37.110566,-43.896218,-55.277679,-77.231510,-4.899449,-81.704037,100.000000,100.000000,-60.917096,99.872532
Maritime,-89.689210,-70.072280,46.427379,70.225032,100.000000,41.189947,93.873600,40.498453,-25.642740,-70.193055,-23.795701,-42.158055


In [13]:
Relative_error_2015=production_based_emissions.groupby(["scenario","region"]).sum().loc["INDC"]
Relative_error_2015_ratio=(Relative_error_2015["2015"]-Relative_error_2015["2015-IMACLIM"])/np.max([Relative_error_2015["2015"],Relative_error_2015["2015-IMACLIM"]],axis=0)
Relative_error_2015_ratio=Relative_error_2015_ratio*100
pd.DataFrame(Relative_error_2015_ratio)

,0
region,
AFR,-42.134653
BRA,11.368137
CAN,-17.008074
CHN,43.149328
CIS,-59.309775
EUR,-6.435027
IND,24.303429
JAN,-30.547136
MDE,18.434667


In [24]:
Relative_error_2015["2015"]+model.iot.stressor_extension.F_Y.sum(axis=1,level=0).loc["CO2"]

region
AFR    1.075643e+12
BRA    4.973495e+11
CAN    5.678739e+11
CHN    9.929628e+12
CIS    1.475975e+12
EUR    4.548643e+12
IND    2.118380e+12
JAN    1.628660e+12
MDE    2.189486e+12
RAL    1.286805e+12
RAS    3.212389e+12
USA    5.145479e+12
dtype: float64

In [11]:
import src.advance
final_data_ratio,final_technical_coef,Link_country,Link,Production_volumes,total_consumption,direct_emissions=src.advance.extract_data("IMACLIM_format")

### Comparing Consumption emission

In [5]:
import src.advance
conso=src.advance.exctract_consumption_emission("IMACLIM_format")

/home/ubuntu/miniforge3/envs/MatMat-Trade-env/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [24]:
model_base_year_CO2_conso=model.iot.stressor_extension.F_Y.groupby("region",axis=1).sum().loc["CO2"]
IMACLIM_base_year_CO2_conso=conso.groupby(["Scenario","Region"]).sum().loc["INDC",2015]*1e9
comparison=pd.DataFrame([model_base_year_CO2_conso,IMACLIM_base_year_CO2_conso]).T
comparison.rename(columns={"CO2":"MRIO",2015:"IMACLIM"},inplace=True)
comparison.loc["World","MRIO"]=comparison.loc[:,"MRIO"].fillna(0).sum()
comparison

,MRIO,IMACLIM
AFR,3.122832e+11,3.058000e+11
BRA,8.498662e+10,1.123300e+11
CAN,1.266648e+11,1.276800e+11
CHN,4.574793e+11,5.431000e+11
CIS,2.479844e+11,5.065000e+11
EUR,9.358370e+11,1.337600e+12
IND,2.191543e+11,1.779100e+11
JAN,2.213855e+11,5.014000e+11
MDE,3.301295e+11,3.159000e+11
RAL,2.712905e+11,3.590000e+11
